<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 6: Advanced Simulation Methods

---

## Table of Contents
* [cadCAD Simulation Methods](#cadCAD-Simulation-Methods)
  1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
  2. [Parameter Sweeps](#2.-Parameter-Sweeps)
  3. [A/B Testing](#3.-A/B-Testing)
* [cadCAD Simulation Methods Overview](#cadCAD-Simulation-Methods-Overview)
* [Data Manipulation & Analysis](#Data-Manipulation-&-Analysis)

---

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} - \epsilon * population_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'reproduction_rate'}\\
\epsilon: \quad &\textrm{'death_rate'}\\
\beta: \quad &\textrm{'consumption_rate'}\\
\gamma: \quad &\textrm{'growth_rate'}\\
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha), and dies at a rate proportional to the population size $\epsilon$ (epsilon).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

<center>
<img src="./images/s6-differential-spec-ecosystem-final.png"
     alt="Diff spec"
     style="width: 60%" />
</center>

Import and run the existing ecosystem model code from section 5:

In [133]:
import math
import pandas as pd
import plotly

pd.options.plotting.backend = "plotly"

from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [134]:
from cadCAD.configuration import Experiment

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()

# cadCAD Simulation Methods

1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
2. [Parameter Sweeps](#2.-Parameter-Sweeps)
3. [A/B testing](#3.-A/B-Testing)

## 1. Monte Carlo Method

> Monte Carlo methods, or Monte Carlo experiments, are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle.

Source: https://en.wikipedia.org/wiki/Monte_Carlo_method

### Non-determinism

> In computer science, a nondeterministic algorithm is an algorithm that, even for the same input, can exhibit different behaviors on different runs, as opposed to a deterministic algorithm.

Source: https://en.wikipedia.org/wiki/Nondeterministic_algorithm

\begin{align}
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma * rand() \quad \textrm{(tons of grass/month)} \\
\end{align}


* Addition of a random food growth rate, using `rand()`.

In [135]:
from numpy import random

In [136]:
random.rand()

0.1782247616831507

In [137]:
# Setting a seed so that we can reproduce the experiment
random.seed(33)

In [138]:
# With a given seed, generates a random number from a given set
[
    random.rand(),
    random.rand()
]

[0.24851012743772993, 0.44997542105079547]

In [139]:
random.seed(33)
[
    random.rand(),
    random.rand()
]

[0.24851012743772993, 0.44997542105079547]

In [140]:
random.seed(None)

In [141]:
seeds = [
    random.RandomState(1),
    random.RandomState(2),
    random.RandomState(3),
]

In [142]:
[
    seeds[0].rand(),
    seeds[1].rand(),
    seeds[2].rand(),
]

[0.417022004702574, 0.43599490214200376, 0.5507979025745755]

In [143]:
MONTE_CARLO_RUNS = 2

seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)]
seeds

[RandomState(MT19937) at 0x7FD5D40E8540,
 RandomState(MT19937) at 0x7FD5E3057740]

In [144]:
initial_state = {
    'population': 750, # number of sheep
    'food': 14000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.4],
    'death_rate': [0.02],
    'consumption_rate': [0.43],
    'growth_rate': [23.0],
}

In [145]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [146]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_growth(params, substep, state_history, previous_state):
    run = previous_state['run']
    food_growth = params['growth_rate'] * seeds[run - 1].rand()
    return {'delta_food': food_growth}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

In [147]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [148]:
SIMULATION_TIMESTEPS = 500

In [149]:
sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config,
)

In [150]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (500, 4, 2, 2)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 0]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
Total execution time: 0.08s


In [151]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4
...,...,...,...,...,...,...,...
997,49,0.362806,0,0,2,1,496
998,49,0.840778,0,0,2,1,497
999,49,0.000000,0,0,2,1,498
1000,49,0.094750,0,0,2,1,499


In [152]:
df = simulation_result.copy()
df = df[df.simulation == 0]
df

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4
...,...,...,...,...,...,...,...
997,49,0.362806,0,0,2,1,496
998,49,0.840778,0,0,2,1,497
999,49,0.000000,0,0,2,1,498
1000,49,0.094750,0,0,2,1,499


In [153]:
df[df.run == 1].head()

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4


In [154]:
df[df.run == 2].head()

,population,food,simulation,subset,run,substep,timestep
501,750,14000.000000,0,0,2,0,0
502,6335,13687.091506,0,0,2,1,1
503,11684,10979.608969,0,0,2,1,2
504,15843,5955.491600,0,0,2,1,3
505,17909,0.000000,0,0,2,1,4


In [155]:
df[df.run == 3].head()

,population,food,simulation,subset,run,substep,timestep


### Simulation Analysis

In [100]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='run', height=800, width=1000)

## 2. Parameter Sweeps

In [156]:
dictionary = {
    'reproduction_rate': [0.1, 0.2],
    'death_rate': [0.01, 0.02]
}
dictionary['reproduction_rate'][0]

0.1

In [157]:
nested_dictionary = {
    'population': [
        {'reproduction_rate': 0.1, 'death_rate': 0.01},
        {'reproduction_rate': 0.2, 'death_rate': 0.02},
    ]
}
nested_dictionary['population'][0]['reproduction_rate']

0.1

In [158]:
initial_state = {
    'population': 750, # number of sheep
    'food': 10000 # tons of grass
}

system_params = {
    'population': [
        {'reproduction_rate': 0.2, 'death_rate': 0.05},
        {'reproduction_rate': 0.4, 'death_rate': 0.02},
        {'reproduction_rate': 0.8, 'death_rate': 0.002}
    ],
    'food': [
        {'consumption_rate': 0.08, 'growth_rate': 7.0},
        {'consumption_rate': 0.02, 'growth_rate': 12.0},
        {'consumption_rate': 0.005, 'growth_rate': 40.0}
    ]
}

In [159]:
{
    'population': system_params['population'][0],
    'food': system_params['food'][0]
}

{'population': {'reproduction_rate': 0.2, 'death_rate': 0.05},
 'food': {'consumption_rate': 0.08, 'growth_rate': 7.0}}

In [160]:
{
    'population': system_params['population'][1],
    'food': system_params['food'][1]
}

{'population': {'reproduction_rate': 0.4, 'death_rate': 0.02},
 'food': {'consumption_rate': 0.02, 'growth_rate': 12.0}}

In [161]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [178]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['population']['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['population']['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_consumption(params, substep, state_history, previous_state):
    run = previous_state['run']
    food_consumption = params['food']['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}
    
def p_growth(params, substep, state_history, previous_state):
    food_growth = params['food']['growth_rate']
    return {'delta_food': food_growth}

In [179]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [180]:
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 500

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

In [181]:
from pprint import pprint

print('system_params')
pprint(system_params)
print(' ')
print('sim_config')
pprint(sim_config)

system_params
{'food': [{'consumption_rate': 0.08, 'growth_rate': 7.0},
          {'consumption_rate': 0.02, 'growth_rate': 12.0},
          {'consumption_rate': 0.005, 'growth_rate': 40.0}],
 'population': [{'death_rate': 0.05, 'reproduction_rate': 0.2},
                {'death_rate': 0.02, 'reproduction_rate': 0.4},
                {'death_rate': 0.002, 'reproduction_rate': 0.8}]}
 
sim_config
[{'M': {'food': {'consumption_rate': 0.08, 'growth_rate': 7.0},
        'population': {'death_rate': 0.05, 'reproduction_rate': 0.2}},
  'N': 1,
  'T': range(0, 500)},
 {'M': {'food': {'consumption_rate': 0.02, 'growth_rate': 12.0},
        'population': {'death_rate': 0.02, 'reproduction_rate': 0.4}},
  'N': 1,
  'T': range(0, 500)},
 {'M': {'food': {'consumption_rate': 0.005, 'growth_rate': 40.0},
        'population': {'death_rate': 0.002, 'reproduction_rate': 0.8}},
  'N': 1,
  'T': range(0, 500)}]


In [182]:
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [183]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 3
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (500, 4, 2, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 1, 1, 1, 2, 2, 2]
SubsetIDs: [0, 0, 0, 1, 2, 0, 1, 2]
Ns       : [0, 1, 0, 1, 2, 0, 1, 2]
ExpIDs   : [0, 0, 1, 1, 1, 2, 2, 2]
Execution Mode: parallelized
Total execution time: 0.18s


In [184]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,88,21.320000,2,0,1,1,496
3002,88,21.280000,2,0,1,1,497
3003,88,21.240000,2,0,1,1,498
3004,88,21.200000,2,0,1,1,499


In [185]:
df = simulation_result.copy()
df = df[df.simulation == 1]
df

,population,food,simulation,subset,run,substep,timestep
1002,750,10000.00,1,0,1,0,0
1003,2713,9947.00,1,0,1,1,1
1004,4567,9736.96,1,0,1,1,2
1005,6287,9378.60,1,0,1,1,3
1006,7849,8882.64,1,0,1,1,4
...,...,...,...,...,...,...,...
2500,53974,0.00,1,2,3,1,496
2501,53867,0.00,1,2,3,1,497
2502,53760,0.00,1,2,3,1,498
2503,53653,0.00,1,2,3,1,499


### Simulation Analysis

In [186]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='subset', height=800, width=1000)

## 3. A/B Testing

In [187]:
df = simulation_result.copy()
df

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,88,21.320000,2,0,1,1,496
3002,88,21.280000,2,0,1,1,497
3003,88,21.240000,2,0,1,1,498
3004,88,21.200000,2,0,1,1,499


In [188]:
import plotly.express as px

fig = px.line(
    df,
    x='timestep',
    y=['population', 'food'],
    facet_row='simulation',
    facet_col='run',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

# cadCAD Simulation Methods Overview

<center>
<img src="./images/cadcad-simulation-methods.png"
     alt="Simulation methods"
     style="width: 600px;" />
</center>

# Data Manipulation & Analysis

In [189]:
df = simulation_result.copy()
df

,population,food,simulation,subset,run,substep,timestep
0,750,14000.000000,0,0,1,0,0
1,6335,13690.122711,0,0,1,1,1
2,11685,10982.522066,0,0,1,1,2
3,15845,5971.835624,0,0,1,1,3
4,17917,0.000000,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,88,21.320000,2,0,1,1,496
3002,88,21.280000,2,0,1,1,497
3003,88,21.240000,2,0,1,1,498
3004,88,21.200000,2,0,1,1,499


In [190]:
df = df.query("simulation == 1 and run == 2")
df

,population,food,simulation,subset,run,substep,timestep
1503,750,10000.00,1,1,2,0,0
1504,4735,9997.00,1,1,2,1,1
1505,8640,9914.30,1,1,2,1,2
1506,12433,9753.50,1,1,2,1,3
1507,16086,9516.84,1,1,2,1,4
...,...,...,...,...,...,...,...
1999,624,27.78,1,1,2,1,496
2000,623,27.30,1,1,2,1,497
2001,622,26.84,1,1,2,1,498
2002,621,26.40,1,1,2,1,499


In [191]:
fig = px.scatter(
    df[df.timestep >= 80],
    x='population',
    y='food',
    color='timestep'
)
fig.show()

<br/><br/><br/>
# Well done!
You're now well on your way to being a cadCAD system modeller!
<br/><br/><br/><br/>